In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification
# from peft import PeftModel
import torch
import numpy as np

In [2]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Обученная

In [ ]:
# checkpath = 'qwen2.5-lora-jokes/v2.0/checkpoint-764_scoutie' #scoutie dataset
checkpath = 'qwen2.5-lora-jokes/v2.0/checkpoint-2906_parsed' # parsed dataset
model = AutoModelForCausalLM.from_pretrained(checkpath, trust_remote_code=True)
# model = PeftModel.from_pretrained(base_model, checkpath)
# model = model.merge_and_unload()
model.eval()


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
/home/t3fio/miniconda3/envs/jokes/lib/python3.13/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

# Необученная

In [4]:

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     device_map="auto",
#     torch_dtype="auto",
# )


# Получение анекдотов

In [ ]:
prompt = "Заходит мужик в шкаф"
messages = [
    {"role": "system", "content": "Ты генератор шуток, отвечай только с помощью анекдотов."},
    {"role": "user", "content": "Заходит бесконечное количество математиков в бар"},
    {"role": "assistant", "content": "Бесконечное число математиков заходит в бар. Первый говорит: «Мне кружку пива!». Второй говорит: «Мне половину кружки пива!» Третий говорит: «Мне четверть кружки пива!» Четвертый говорит: «Мне 1/8 кружки пива!» Бармен:– Да знаю я вас – вам две кружки на всех!"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=256,
    num_return_sequences = 5, # Количество ответов
)

anecdotes = [prompt + '\n' + tokenizer.batch_decode([
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, [generated_ids[i]])
    ], skip_special_tokens=True)[0] for i in range(len(generated_ids))]

In [ ]:
# Загрузка модели
bert = DistilBertForSequenceClassification.from_pretrained('funny_jokes_classifier.model')
bert_tokenizer = DistilBertTokenizer.from_pretrained('funny_jokes_classifier.tokenizer')
MAX_LEN= 128
# Предсказание на новом тексте
def predict_funny_score(model, joke, tokenizer):
    model.to('cpu')
    # Токенизация
    encoded_joke = tokenizer.encode_plus(
        joke,
        max_length=MAX_LEN,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoded_joke['input_ids'].to('cpu')
    attention_mask = encoded_joke['attention_mask'].to('cpu')

    # Предсказание
    with torch.no_grad():
        output = model(input_ids, attention_mask)

    # Получаем сырое предсказание
    prediction = output.logits.squeeze()

    return 1 / (1 + np.exp(-prediction))

In [10]:
for anecdote in anecdotes:
    score = predict_funny_score(bert, anecdote, bert_tokenizer)
    print(f'Шутка: "{anecdote[:]}..."\nОценка смешнявости: {score:.2f}\n')

/tmp/ipykernel_9479/2634738824.py:30: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  return 1 / (1 + np.exp(-prediction))


Шутка: "Заходит мужик в шкаф
Мужик вошел в шкаф и увидел там лампочку...."
Оценка смешнявости: 0.46

Шутка: "Заходит мужик в шкаф
Заводской номер 2345..."
Оценка смешнявости: 0.41

Шутка: "Заходит мужик в шкаф
Здесь нет ни одной женщины!..."
Оценка смешнявости: 0.42

Шутка: "Заходит мужик в шкаф
Кто-нибудь может объяснить мне, почему я не могу найти свою жену?..."
Оценка смешнявости: 0.56

Шутка: "Заходит мужик в шкаф
Мужик вошел в шкаф. Навстречу ему стоит женщина. Мужик говорит: - Привет, ты что там делаешь? Жена: - Я твоя жена. Мужик: - А мне какая она?..."
Оценка смешнявости: 0.60

